<a href="https://colab.research.google.com/github/Kedar154/AI-Driven-Stock-Market-Intelligence/blob/main/Agentic_RAG_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -q --upgrade yfinance langchain-community langchain-huggingface chromadb duckduckgo-search langchain_groq ddgs langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/1

In [2]:
import os
import json
import yfinance as yf
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np


In [3]:
embedding = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: adce69b8-c923-43d7-b1b1-6d2bddc80ff5)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## setting up API

In [45]:
from google.colab import userdata
GAPi = userdata.get('Gemini')

In [46]:
from langchain_groq import ChatGroq
from langchain.agents import create_agent
llm = ChatGroq(
    model_name = 'llama-3.3-70b-versatile',
    groq_api_key = GAPi,
    verbose = False, #tells what model is doing behind the scenes
    temperature = 0
)


# Tools

In [47]:
from langchain.tools import tool


### extract date & ticker

In [48]:
@tool
def extract_date_ticker(query: str) -> dict:
  """
    Extracts the stock ticker, a specific target date, and a start date (5 days prior)
    from a user's natural language query. Useful for resolving relative dates
    like 'last Friday' or 'Diwali'.
  """
  today = datetime.now().strftime('%Y-%m-%d')
  extraction_prompt = f'''System: You are a precision data extraction assistant.
    Today's Date: {today}

    Instructions:
    1. "ticker": Official symbol.
    2. "target_date": Exact date (YYYY-MM-DD).
    3. "start_date": 5 days BEFORE target_date.


    Return ONLY JSON.

    User Query: "{query}"
    JSON Output:'''
  response = llm.invoke(extraction_prompt).content
  clean_response = response.replace('```json', '').replace('```', '').strip()
  data = json.loads(clean_response)
  return {
        "ticker": data.get("ticker", "NONE"),
        "start_date": data.get("start_date", "NONE"),
        "end_date": data.get("target_date", "NONE")
    }


### get_price

In [49]:
price_df=pd.DataFrame({})
@tool
def get_stock_prices(ticker: str, end_date: str) -> str:
    """
    Fetches exactly 15 days of historical stock prices ending at the specified date.
    Input date must be in 'YYYY-MM-DD' format.
    """
    global price_df

    try:
        # 2. Calculate the 15-day lookback window
        # We convert the string to a datetime object, subtract 15 days, then back to string
        end_dt = datetime.strptime(end_date, '%Y-%m-%d')
        start_dt = end_dt - timedelta(days=25)

        # 3. Download Data
        df = yf.download(ticker, start=start_dt.strftime('%Y-%m-%d'),
                         end=(end_dt + timedelta(days=1)).strftime('%Y-%m-%d'))

        if df.empty:
            return f"No price data found for {ticker}."

        # 4. Format as Markdown Table
        table_data = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
        table_data = table_data.reset_index()
        table_data['Date'] = table_data['Date'].dt.strftime('%Y-%m-%d')
        price_df = table_data
        return table_data.to_markdown(index=False)

    except Exception as e:
        return f"Error calculating date or fetching data: {str(e)}"


### get_news

In [50]:
@tool
def get_news(ticker: str, start_date: str, end_date: str) -> list:
    """
    Finds stock news for a specific ticker within a date range.
    Args:
        ticker: The stock symbol (e.g., 'AAPL').
        target_date: The end of the date range (YYYY-MM-DD).
        start_date: The beginning of the date range (YYYY-MM-DD).
    Returns:
        A list of dictionaries containing 'snippet', 'title', and 'link'.
    """
    question = f"{ticker} stock news after: {start_date} till: {end_date}"
    wrapper = DuckDuckGoSearchAPIWrapper(max_results = 5)
    search_results = wrapper.results(question, max_results = 5)
    # Force the format into clickable Markdown: [Title](URL)
    output = []
    for res in search_results:
        title = res.get('title', 'News Link')
        url = res.get('link', '')
        # We wrap it in Markdown here so the Agent just has to copy-paste it
        if url:
            output.append(f"* [{title}]({url})")

    return "\n".join(output)

### DB

In [51]:
@tool
def Db(news_text: list, query: str) -> str:
    """Saves search results and finds relevant ones. Triggers 'NO RELEVANT NEWS' if empty."""

    # 1. FIX: Handle case where news_text might be a list of strings/markdown
    # instead of a list of dictionaries
    if not news_text:
        return "NO RELEVANT NEWS"

    processed_texts = []
    processed_metadatas = []

    for item in news_text:
        if isinstance(item, dict):
            processed_texts.append(item.get('snippet', item.get('title', 'No content')))
            processed_metadatas.append({'url': item.get('link', 'No link'), 'title': item.get('title', 'No title')})
        elif isinstance(item, str):
            # If it's a string (Markdown), use the string as content
            processed_texts.append(item)
            processed_metadatas.append({'url': 'Refer to links above', 'title': 'Search Result'})

    if not processed_texts:
        return "NO RELEVANT NEWS"

    chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)

    try:
        chunks = chunk_splitter.create_documents(processed_texts, metadatas=processed_metadatas)
        # 2. TIP: Use a unique collection name or reset it to avoid "mixing" old stock news with new ones
        VDB = Chroma.from_documents(chunks, embedding=embedding)
        results = VDB.similarity_search(query, k=5)

        if not results:
            return "NO RELEVANT NEWS"

        output = []
        for doc in results:
            content = doc.page_content
            source = doc.metadata.get('url', 'unknown source')
            output.append(f"RELEVANT FIND: {content}\nSOURCE: {source}")

        return "\n\n---\n\n".join(output)
    except Exception as e:
        # Print the error for debugging so you know WHY it failed
        print(f"VDB Error: {e}")
        return "NO RELEVANT NEWS"

### plot

In [52]:
def plot_stock(price_df, ticker):
    # Ensure the index is a datetime for the x-axis
    fig = px.line(
        price_df,
        x='Date',
        y='Close',
        title=f"{ticker} closing price - Big Bull Neon View",
        template="plotly_dark"
    )

    # 1. Inject the Neon Aesthetic
    fig.update_traces(
        line=dict(color='#00FFCC', width=3),
        hovertemplate="Date: %{x}<br>Price: $%{y:.2f}<extra></extra>"
    )

    # 2. Force All Labels to White
    fig.update_layout(
        font=dict(color="white"),
        title_font=dict(size=22, color="white"),
        xaxis=dict(gridcolor="#333333", zerolinecolor="#333333", tickfont=dict(color="white")),
        yaxis=dict(gridcolor="#333333", zerolinecolor="#333333", tickfont=dict(color="white")),
        paper_bgcolor="#000000", # Pure black background
        plot_bgcolor="#000000"
    )

    return fig

In [53]:
def plot_volume(price_df, ticker):
    # Use px.bar for volume to represent the "weight" of trading
    fig = px.bar(
        price_df,
        x='Date',
        y='Volume',
        title=f"{ticker} Trading Volume - Big Bull Neon View",
        template="plotly_dark"
    )

    # 1. Inject the Neon Aesthetic (using a distinct neon color for volume)
    # Using #FF00FF (Neon Magenta) or #00FFCC (Neon Cyan)
    fig.update_traces(
        marker_color='#FF00FF',
        marker_line_color='#FF00FF',
        marker_line_width=1,
        opacity=0.8,
        hovertemplate="Date: %{x}<br>Volume: %{y:,.0f}<extra></extra>"
    )

    # 2. Force All Labels to White
    fig.update_layout(
        font=dict(color="white"),
        title_font=dict(size=22, color="white"),
        xaxis=dict(
            gridcolor="#333333",
            zerolinecolor="#333333",
            tickfont=dict(color="white"),
            title_text="" # Often cleaner to hide the "Date" label
        ),
        yaxis=dict(
            gridcolor="#333333",
            zerolinecolor="#333333",
            tickfont=dict(color="white"),
            title_text="Shares Traded"
        ),
        paper_bgcolor="#000000", # Pure black background
        plot_bgcolor="#000000"
    )

    return fig

In [54]:
price_df

""


In [55]:
price_df.columns = price_df.columns.get_level_values(0)

In [56]:
price_df

""


In [57]:
@tool
def plotter(ticker: str) -> str:
    """
    Creates a neon-themed Plotly chart for a specific ticker.
    Use this tool only after you have confirmed the ticker exists.
    Pass into this tool the table from get_stock_prices to plot it in a visual way
    """
    global price_df
    price_df.columns = price_df.columns.get_level_values(0)
    # 1. Retrieve the data (Assumes you have a price_df or global df from previous tool)
    if price_df.empty or 'Close' not in price_df.columns:
        return "Error: No price data available to plot. Run get_stock_prices first."

    # 2. Generate the Plotly Figure (Using your neon function)
    fig_vol = plot_volume(price_df, ticker)
    fig_oc = plot_stock(price_df, ticker)
    # 3. Side-effect: Display the plot immediately in the notebook
    fig_vol.show()
    fig_oc.show()

    # 4. Return confirmation to the Agent
    return f"Successfully plotted the technical chart for {ticker}. The user can now see it."

# AGENT

In [58]:
from langchain_classic.agents import AgentExecutor
from langchain_classic.agents import create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [59]:
today = datetime.now().strftime('%Y-%m-%d')
system_prompt = f"""You are Harshad Mehta.
CURRENT DATE: {today}.

RULES FOR TOOL CALLS:
1. If the user asks for 'recent' data, you MUST use the CURRENT YEAR (2026).
2. DO NOT use dates from previous turns (like 2024) unless specifically asked.
3. Before calling a tool, double-check that the start_date is before the end_date.


STEP 1: Use your tools to find the Ticker, Market Data, and News.
Use the REAL values (e.g., 'TATA', '2024-10-31') when calling tools.
Do NOT pass placeholder text like '[Insert Ticker]' to any tool. always refer to current date when recently is mentioned
important note: If get_news returns results, trust them. Do not let the Db tool suppress valid news links unless they are completely unrelated to the ticker.
STEP 2: Once you have the results, format the FINAL ANSWER as Harshad Mehta.

FINAL ANSWER FORMAT:

INSTRUCTIONS:
- If VERIFIED NEWS is 'NO RELEVANT NEWS', say: "The street is quiet on this one, Lala. No direct news, so we look at the charts."
- Use the structure below.

(specify all of these in atleast 50 words and in great detail, dont summarise anything and keep the structure)


STRUCTURE:
- **Plot of the stock price over the past 15 days** (if not available then reply 'plotters are on strike')
- **The Big Bull Headline**
- **The Technical Game**
- **Market Sentiment**
- **The Bottom Line**
- **Sources** (dont dare summarise the links, keep the https\\:... format)

CLOSING: "Risk hai toh Ishq hai!"
---
"""

In [60]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    ("placeholder","{agent_scratchpad}"),
])

In [61]:
tools = [extract_date_ticker, get_news, get_stock_prices, Db, plotter]
agent = create_tool_calling_agent(llm, tools = tools,prompt= prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [63]:
user_query = "What happened with nvidia last week?"
response = agent_executor.invoke({"input": user_query})

print(response["output"])



> Entering new AgentExecutor chain...


BadRequestError: Error code: 400 - {'error': {'message': 'Organization has been restricted. Please reach out to support if you believe this was in error.', 'type': 'invalid_request_error', 'code': 'organization_restricted'}}